In [1]:
pip install tensorflow-gpu==2.8.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install protobuf==3.20

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import warnings
# 경고메세지 끄기
warnings.filterwarnings (action='ignore')

In [9]:
import pandas as pd
df = pd.read_csv('./우석_정규표현식.csv')

In [10]:
df.head()

,labels,동영상 제목,댓글,댓글 작성일,PRO
0,,트랜스픽션 피파온라인 BGM 모아듣기 I 피파온라인4 구단주분들을 위한 온라인 콘서...,00:34 너를 원해\r\n03:46 Radio\r\n07:17 승리를 위하여\r\...,2021-07-31T06:24:24Z,00 34 너를 원해 03 46 Radio 07 17 승리를 위하여 10 26...
1,,트랜스픽션 피파온라인 BGM 모아듣기 I 피파온라인4 구단주분들을 위한 온라인 콘서...,아직도 이노래를 들으면 중학생때 생각이나고 그때 주변환경 친구들 추억이 생각난다.....,2023-11-02T23:57:44Z,아직도 이노래를 들으면 중학생때 생각이나고 그때 주변환경 친구들 추억이 생각난다 ...
2,,트랜스픽션 피파온라인 BGM 모아듣기 I 피파온라인4 구단주분들을 위한 온라인 콘서...,내 인생 최고의 온라인게임은 바로 피파온라인2,2023-11-02T19:00:05Z,내 인생 최고의 온라인게임은 바로 피파온라인2
3,,트랜스픽션 피파온라인 BGM 모아듣기 I 피파온라인4 구단주분들을 위한 온라인 콘서...,난 피파를 안하는데 왜 이 노래들을 아는거지???,2023-11-02T14:12:40Z,난 피파를 안하는데 왜 이 노래들을 아는거지
4,,트랜스픽션 피파온라인 BGM 모아듣기 I 피파온라인4 구단주분들을 위한 온라인 콘서...,와 소름 돋는다 진짜 ㅠ 감동이요감동,2023-11-01T04:57:14Z,와 소름 돋는다 진짜 ㅠ 감동이요감동


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252989 entries, 0 to 252988
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   labels  252985 non-null  object
 1   동영상 제목  252989 non-null  object
 2   댓글      252989 non-null  object
 3   댓글 작성일  252989 non-null  object
 4   PRO     252989 non-null  object
dtypes: object(5)
memory usage: 9.7+ MB


## 긍정,부정 분류 모델 적용

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import time
import re
import os
import urllib.request
from tqdm import tqdm

In [17]:
# GPU 사용 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"사용하고 있는 건? → {device}")

사용하고 있는 건? → cuda


In [18]:
# 모델 불러오기
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", from_pt=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

# 가중치 저장
model.save_weights("./best_model.h5")

# 가중치 로드
model.load_weights("./best_model.h5")

2023-11-21 02:16:11.623466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 02:16:11.627085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 02:16:11.628983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 02:16:11.632863: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [21]:
# 댓글 예측 함수 정의
def predict_comments(df):
    inputs = tokenizer(df["댓글"].tolist(), return_tensors='tf', truncation=True, max_length=256, padding=True)
    inputs_dict = {key: inputs[key].numpy() for key in inputs}
    outputs = model.predict(inputs_dict)
    predicted_labels = tf.argmax(tf.nn.softmax(outputs["logits"], axis=1), axis=1)
    return predicted_labels.numpy()

In [22]:
# 진행상황 출력
def print_progress(count, total, start_time):
    percent = count / total * 100
    elapsed_time = time.time() - start_time
    print(f"{count} / {total}({percent:.2f}%) ({elapsed_time:.2f} seconds)")

# 댓글을 500개씩 split
df_split = np.array_split(df, len(df) // 500 + 1)

# 진행시간 측정
start_time = time.time()

# 각 split된 데이터프레임의 댓글 예측
predictions_list = []
for i, df_split_item in enumerate(df_split):
    predictions_list.append(predict_comments(df_split_item))
    print_progress(i + 1, len(df_split), start_time)

# 예측 결과 합치기
predictions_df = np.concatenate(predictions_list)

# 예측 결과 출력
print("댓글 예측 결과:", predictions_df)

1 / 506(0.20%) (13.13 seconds)
2 / 506(0.40%) (21.08 seconds)
3 / 506(0.59%) (29.05 seconds)
4 / 506(0.79%) (37.07 seconds)
5 / 506(0.99%) (45.09 seconds)
6 / 506(1.19%) (53.16 seconds)
7 / 506(1.38%) (61.26 seconds)
8 / 506(1.58%) (69.35 seconds)
9 / 506(1.78%) (77.59 seconds)
10 / 506(1.98%) (85.70 seconds)
11 / 506(2.17%) (93.85 seconds)
12 / 506(2.37%) (102.00 seconds)
13 / 506(2.57%) (110.17 seconds)
14 / 506(2.77%) (118.33 seconds)
15 / 506(2.96%) (126.48 seconds)
16 / 506(3.16%) (134.63 seconds)
17 / 506(3.36%) (142.84 seconds)
18 / 506(3.56%) (151.04 seconds)
19 / 506(3.75%) (159.24 seconds)
20 / 506(3.95%) (167.44 seconds)
21 / 506(4.15%) (175.64 seconds)
22 / 506(4.35%) (183.83 seconds)
23 / 506(4.55%) (192.06 seconds)
24 / 506(4.74%) (200.27 seconds)
25 / 506(4.94%) (208.50 seconds)
26 / 506(5.14%) (216.68 seconds)
27 / 506(5.34%) (224.85 seconds)
28 / 506(5.53%) (233.01 seconds)
29 / 506(5.73%) (241.16 seconds)
30 / 506(5.93%) (249.34 seconds)
31 / 506(6.13%) (257.50 second

237 / 506(46.84%) (1876.68 seconds)
238 / 506(47.04%) (1884.88 seconds)
239 / 506(47.23%) (1893.06 seconds)
240 / 506(47.43%) (1899.28 seconds)
241 / 506(47.63%) (1907.46 seconds)
242 / 506(47.83%) (1915.64 seconds)
243 / 506(48.02%) (1923.81 seconds)
244 / 506(48.22%) (1932.00 seconds)
245 / 506(48.42%) (1940.20 seconds)
246 / 506(48.62%) (1948.40 seconds)
247 / 506(48.81%) (1956.60 seconds)
248 / 506(49.01%) (1964.78 seconds)
249 / 506(49.21%) (1970.03 seconds)
250 / 506(49.41%) (1978.23 seconds)
251 / 506(49.60%) (1986.41 seconds)
252 / 506(49.80%) (1994.38 seconds)
253 / 506(50.00%) (2002.56 seconds)
254 / 506(50.20%) (2010.70 seconds)
255 / 506(50.40%) (2018.89 seconds)
256 / 506(50.59%) (2027.14 seconds)
257 / 506(50.79%) (2035.33 seconds)
258 / 506(50.99%) (2043.52 seconds)
259 / 506(51.19%) (2051.75 seconds)
260 / 506(51.38%) (2059.96 seconds)
261 / 506(51.58%) (2067.70 seconds)
262 / 506(51.78%) (2075.91 seconds)
263 / 506(51.98%) (2084.15 seconds)
264 / 506(52.17%) (2092.39 s

465 / 506(91.90%) (3639.91 seconds)
466 / 506(92.09%) (3648.08 seconds)
467 / 506(92.29%) (3653.22 seconds)
468 / 506(92.49%) (3661.41 seconds)
469 / 506(92.69%) (3669.35 seconds)
470 / 506(92.89%) (3677.55 seconds)
471 / 506(93.08%) (3685.71 seconds)
472 / 506(93.28%) (3693.90 seconds)
473 / 506(93.48%) (3702.07 seconds)
474 / 506(93.68%) (3710.25 seconds)
475 / 506(93.87%) (3718.43 seconds)
476 / 506(94.07%) (3726.61 seconds)
477 / 506(94.27%) (3734.78 seconds)
478 / 506(94.47%) (3742.94 seconds)
479 / 506(94.66%) (3751.15 seconds)
480 / 506(94.86%) (3759.39 seconds)
481 / 506(95.06%) (3767.55 seconds)
482 / 506(95.26%) (3775.74 seconds)
483 / 506(95.45%) (3783.93 seconds)
484 / 506(95.65%) (3792.13 seconds)
485 / 506(95.85%) (3800.33 seconds)
486 / 506(96.05%) (3808.56 seconds)
487 / 506(96.25%) (3816.77 seconds)
488 / 506(96.44%) (3824.97 seconds)
489 / 506(96.64%) (3831.54 seconds)
490 / 506(96.84%) (3839.73 seconds)
491 / 506(97.04%) (3846.45 seconds)
492 / 506(97.23%) (3854.64 s

In [23]:
# 예측 결과를 "긍정" 또는 "부정"으로 변환하는 함수 정의
def convert_to_sentiment(predictions):
    sentiment_labels = ["부정", "긍정"]
    return [sentiment_labels[prediction] for prediction in predictions]

# 예측 결과를 "긍정" 또는 "부정"으로 변환
sentiments_df = convert_to_sentiment(predictions_df)

In [24]:
# 예측 결과를 데이터프레임에 추가
df["예측 감정"] = sentiments_df
df["예측 감정"]

0         긍정
1         긍정
2         긍정
3         긍정
4         긍정
          ..
252984    긍정
252985    긍정
252986    긍정
252987    긍정
252988    긍정
Name: 예측 감정, Length: 252989, dtype: object

In [25]:
df.to_csv('전문가 분석 예측 결과.csv', index=False, encoding='utf-8')